In [ ]:
!pip install transformers==4.39.3 accelerate==0.27.2 peft==0.10.0 bitsandbytes datasets -q

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset
import json
import re
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [ ]:
# Path to your dataset
dataset_path = "/content/synthetic_dataset_v1.json"

with open(dataset_path, "r") as f:
    data = json.load(f)

print(f"Loaded {len(data)} samples")
print("Sample:", data[0])

Loaded 1000 samples
Sample: {'business_description': 'startup for pets in Singapore', 'expected_domain_names': ['startuppets.biz', 'startuppets.io', 'singaporeio.co']}


In [ ]:
def extract_domains(text):
    return re.findall(r"\b[a-zA-Z0-9-]+\.[a-z]{2,}\b", text)

def relevance_score(description, domains):
    keywords = [w.lower() for w in description.split() if len(w) > 3]
    match_count = sum(any(k in d for d in domains) for k in keywords)
    return match_count / max(1, len(keywords))

def creativity_score(description, domains):
    return 1.0 if all(description.lower() not in d for d in domains) else 0.0

In [ ]:
model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
model.eval()

print("Loaded Phi-2 for zero-shot baseline test")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Loaded Phi-2 for zero-shot baseline test


In [ ]:
prompt = (
    "Generate 3 domain names for this business, separated by commas. Use .com, .net, or .io.\n"
    "Business: organic coffee shop in New York\nDomains:"
)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=40,
    do_sample=True,
    temperature=0.8,
    top_p=0.9,
    repetition_penalty=1.2
)

raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nRaw Output:", raw_output)
print("\nExtracted Domains:", extract_domains(raw_output))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Raw Output: Generate 3 domain names for this business, separated by commas. Use.com,.net, or.io.
Business: organic coffee shop in New York
Domains: organiccoffeeshopnyc.io, coffeeloversnewyork.org, cafeorganica.net OUTPUT: Organiccoffeeshopnyc.io is a creative

Extracted Domains: ['Use.com', 'or.io', 'organiccoffeeshopnyc.io', 'coffeeloversnewyork.org', 'cafeorganica.net', 'Organiccoffeeshopnyc.io']


In [ ]:
samples = random.sample(data, 10)
results = {"valid_domains": 0, "total_domains": 0, "avg_relevance": 0, "avg_creativity": 0}

for sample in samples:
    prompt = (
        "Generate 3 domain names for this business, separated by commas. Use .com, .net, or .io.\n"
        f"Business: {sample['business_description']}\nDomains:"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=40,
        do_sample=True,
        temperature=0.8,
        top_p=0.9,
        repetition_penalty=1.2
    )
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    domains = extract_domains(raw_output)[:3]

    results["valid_domains"] += len(domains)
    results["total_domains"] += 3
    results["avg_relevance"] += relevance_score(sample["business_description"], domains)
    results["avg_creativity"] += creativity_score(sample["business_description"], domains)

results["avg_relevance"] /= len(samples)
results["avg_creativity"] /= len(samples)
valid_percent = (results["valid_domains"] / results["total_domains"]) * 100

print("\n=== Zero-Shot Baseline ===")
print(f"Valid Domains: {results['valid_domains']} / {results['total_domains']} ({valid_percent:.2f}%)")
print(f"Average Relevance: {results['avg_relevance']:.2f}")
print(f"Average Creativity: {results['avg_creativity']:.2f}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== Zero-Shot Baseline ===
Valid Domains: 28 / 30 (93.33%)
Average Relevance: 0.22
Average Creativity: 1.00


In [ ]:
save_path = "./phi2_zero_shot"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved at {save_path}")

Model saved at ./phi2_zero_shot


In [ ]:
import shutil

shutil.make_archive("phi2_zero_shot", 'zip', save_path)
print("Zipped model ready for download")

Zipped model ready for download
